In [1]:
from data_cleaning import *

c:\Users\Tyreek ALEXANDER\OneDrive - Northern Caribbean University\Documents\EffectiveXGBoost\SURVEY\data_cleaning.py:19: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag=pd.read_csv(z.open(member_name))


In [2]:
from hyperopt import fmin,tpe,hp,STATUS_OK,Trials
from sklearn.metrics import accuracy_score,roc_auc_score

from typing import Any, Dict, Union
import xgboost as xgb

In [3]:
def hyperparameter_tuning(space: Dict[str,Union[float,int]],
                          Xtrain:pd.DataFrame,
                          ytrain:pd.Series,
                          Xtest:pd.DataFrame,
                          ytest:pd.Series,
                          early_stopping_rounds:int=50,
                          metric:callable=accuracy_score)->Dict[str, Any]:
    int_vals=['max_depth','reg_alpha']
    space={k:(int(val) if k in int_vals else val)
           for k,val in space.items()}
    space['early_stopping_rounds']= early_stopping_rounds
    model = xgb.XGBClassifier(**space)
    evaluation=[(Xtrain,ytrain),(Xtest,ytest)]

    model.fit(Xtrain,ytrain,
              eval_set=evaluation,
              verbose=False)
    pred=model.predict(Xtest)
    score=metric(ytest,pred)
    return{'loss':-score,'status':STATUS_OK,'model':model}

In [ ]:
options={'max_depth':hp.quniform('max_depth',1,8,1),
         'min_child_weight':hp.loguniform('min_child_weight',-2,3),
         'subsample':hp.uniform('subsample',0.5,1),
         'colsample_bytree':hp.uniform('colsample_bytree',0.5,0.1),
         'reg_alpha':hp.uniform('reg_alpha',0,10),
         'reg_lambda':hp.uniform('reg_lambda',1,10),
         'gamma':hp.loguniform('gamma',-10,10),
         'learning_rate':hp.loguniform('learning_rate',-7,0),
         'random_state':42}

In [ ]:
trials=Trials()
best=fmin(fn=lambda space: hyperparameter_tuning(space,Xtrain,ytrain,Xtest,ytest),
          space=options,
          algo=tpe.suggest,
          max_evals=2_000,
          trials=trials)